# The main file of the project.

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import app.var_examination as ve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Background info

The Boston Housing Dataset has been removed from the Scikit-Learn library in December 2022, after decades of its use for teaching purposes. Originally, the dataset comes from a research paper *Hedonic housing prices and the demand for clean air* by D. Harrison and D. Rubinfield, published in 1978. 

The dataset has been removed and its use is now discouraged for several reasons, the main of which is the inclusion of the `B` variable, which is a column whose values are calculated as `B=1000(Bk-063)^2` where `Bk` is, "the proportion of blacks by town." The quadratic formula is motivated by an argument of the authors that race segregation has positive impact on housing prices, 

**The goal of this project is to examine the role of the Bk variable and show the issues with its inclussion in the dataset, as well as show the impact of the variable on standard models that were used to process this data.** 

We state that the inclusion of the variable in this context, and the idea of using race to predict prices without considering it as a tremendous issue, are and always will be unaccpetable.

## Current state

Importing the dataset in the traditional way now throws the following error:

In [ ]:
try:
    bostn = datasets.load_boston()
except Exception as e:
    print(e)

# Obtaining the data "by hand"

The documentation of Scikit-Learn advises users to obtain the dataset in the csv format from its original source. We respect that and then convert the data to a well-formatted Pandas DataFrame. 

In the dataset, there are 14 variables:

- **CRIM**: per capita crime rate by town
- **ZN**: proportion of residential land zoned for lots over 25,000 sq.ft.
- **INDUS**: proportion of non-retail business acres per town
- **CHAS**: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
- **NOX**: nitric oxides concentration (parts per 10 million)
- **RM**: average number of rooms per dwelling
- **AGE**: proportion of owner-occupied units built prior to 1940
- **DIS**: weighted distances to five Boston employment centres
- **RAD**: index of accessibility to radial highways
- **TAX**: full-value property-tax rate per $10,000
- **PTRATIO**: pupil-teacher ratio by town
- **B**: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- **LSTAT**: % lower status of the population
- **MEDV**: Median value of owner-occupied homes in $1000's


In [ ]:
# Import the data as instructed by documentation

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

# Split the data into data and target
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

# Convert the data to a well-formatted dataframe

column_names = [
    "CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE",
    "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"
]

feature_columns = column_names[:-1]  # All columns except the target
target_column = column_names[-1]     # The target column

# Construct the features DataFrame
boston_df = pd.DataFrame(data, columns=feature_columns)

# Add the target to the DataFrame
boston_df[target_column] = target

# Exploration & Cleaning

In [ ]:
boston_df.describe()

## Any missing values?

In [ ]:
boston_df.isnull().sum()

## Any filled in values?

**Check if mean, median, min or max values occur suspisiously frequently in the dataset.**

Values in the dataset are rounded to 6 decimal places, match that.

In [ ]:
statistics = []

# The obtained values are rounded to 6 decimal places, match that with the statistics

for column in boston_df.columns:
    mean = boston_df[column].mean().round(6)
    median = boston_df[column].median().round(6)
    mode = boston_df[column].mode().round(6)
    min_value = boston_df[column].min().round(6)
    max_value = boston_df[column].max().round(6)

    mean_frequency = (boston_df[column].value_counts().get(mean, 0) / len(boston_df)) * 100
    median_frequency = (boston_df[column].value_counts().get(median, 0) / len(boston_df)) * 100
    mode_frequency = (boston_df[column].value_counts().get(mode[0], 0) / len(boston_df)) * 100
    min_frequency = (boston_df[column].value_counts().get(min_value, 0) / len(boston_df)) * 100
    max_frequency = (boston_df[column].value_counts().get(max_value, 0) / len(boston_df)) * 100

    statistics.append({
        'Column': column,
        'Mean': f"{mean_frequency:.2f}%",
        'Median': f"{median_frequency:.2f}%",
        'Mode': f"{mode_frequency:.2f}%",
        'Min': f"{min_frequency:.2f}%",
        'Max': f"{max_frequency:.2f}%"
    })

statistics_df = pd.DataFrame(statistics)
print(statistics_df)

A few things arise:

- **ZN** column is mostly zeroes. That makes sense as it is the proportion of residential land zoned for lots over 25,000 sq.ft.
- The high but different percentages of median and max in **RAD** are strange. 
- **B** variable is likely very skewed to the right.

- The mode of 26.09% occurs far too freqeuently.

## Any duplicates?

In [ ]:
duplicates = boston_df.duplicated()
print(duplicates.sum())

## What about the distributions?

In [ ]:
boston_df.hist(figsize=(12, 10), bins=50, grid=False)
plt.tight_layout()
plt.show()

The **RAD** distribution is weird. Noted.

**TAX** is one of the variables that have mode in 26.09% of rows. In this case, it could indicate that most households have a common tax rate, which is alright. **PTRATIO**, **INDUS**, **RAD** and **B** also have modes at or very close to 26.09%. This could indicate that there was a densly populated area that caused such significant values to occur.





## The LSTAT Variable

One of the problematic variables is LSTAT, a percentage of "lower status population". The original paper describes it as 

> Proportion of population that is lower status = 1/2 (proportion of adults without some high education and proportion of male workers classified as laborers)

And the authors suggest that "the effect on price is higher in the upper brackets of society", and therefore a logarithmic transform should be applied in the models.

The sole inclusion and consideration of this variable does raise concerns. For now, we will simply explore its properties and pay more attention to it later on. 

In [ ]:
ve.var_examination(boston_df, "LSTAT")

## The B Variable

A key interest of this project is the B variable. We do not plan on commenting neither the reasons for its inclusion both in the original research paper and in the dataset, nor why the issues were recognized only a few years ago. We will try to examine the variable and later show its role in the dataset.

In the original research paper, the variable is described as 

> Black proportion of population. At low or moderate levels of B, an increase of B should have negative on housing values if Blacks are regarded as undesirable by Whites. However, market discrimination means that market prices are higher at very high levels of B. One expects, therefore, a parabolic relationship between proportion Black in a neighborhood and housing values.

We should note that the above **definition is wrong on many levels** and should not be considered at all in a real world scenario.

In [ ]:
ve.var_examination(boston_df, "B")

We question the need to use the non-linear transform of `B=1000(Bk-0.63)^2`. Therefore, an attempt to extract the original value `Bk` is conducted:

In [ ]:
Bk = np.linspace(0, 1, 100)
Bf = 1000 * (Bk - 0.63) ** 2

plt.plot(Bk, Bf)
plt.xlabel('Bk')
plt.ylabel('Bf')
plt.title('Bf = 1000(Bk-0.63)^2')
plt.grid(True)
plt.show()

## Linear regression model

In following, we will attmpt to train linear regression model capable of predicting value of housing. 

In [ ]:
X = boston_df.drop('MEDV', axis=1)
Y = boston_df['MEDV']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8, random_state=42)


model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

plt.scatter(Y_test, Y_pred)
plt.xlabel('Actual Prices (MEDV)')
plt.ylabel('Predicted Prices (MEDV)')
plt.title('Actual Prices vs. Predicted Prices')
plt.show()